In [58]:
import pandas as pd
import numpy as np
import glob
import re
import networkx as nx
from collections import defaultdict
import itertools
import matplotlib.pyplot as plt
import json
import requests
import gzip
from bs4 import BeautifulSoup
import re
import os

%load_ext autoreload
%autoreload 2
import lib.build_graph as bg
import lib.create_database as cdb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# get files and data
# cfg = json.load(open('config/n_app.json'))
# db.main(**cfg)

In [ ]:
def main(smali_src, number_of_smali, **kwargs):
    # initalize data
    smali = cdb.get_smali(smali_src)
#     test = smali[:number_of_smali]
    
#     calls = list()

    for f in smali:
        blocks = defaultdict(list)
        same_block = defaultdict(list)
        
        with open(f) as file:
            # record all api calls
#             bg.find_api_calls(file, calls)
#             file.seek(0)
            
            # build B
            bg.locate_method_blocks(file, blocks)
            for b in blocks:
                apis = []
                bg.find_api_calls(blocks[b], apis)
                same_block[b] = apis
            bg.create_edges(same_block, B_graph)
            
            file.seek(0)
            # build P
            for line in file:
                if('invoke-' in line and 'method' not in line):
                    package = bg.find_package(line)
                    method = bg.find_method(line)
                    packageD[package].append(method)
                    
                    api = []
                    bg.find_api_calls([line], api)
                    invoke = bg.find_invoke(line)
                    invokeD[invoke].append(api[0])

In [2]:
%%time

# B_graph = nx.Graph()
# P_graph = nx.Graph()
# I_graph = nx.Graph()

# packageD = defaultdict(list)
# invokeD = {'direct': [], 'virtual':[], 'static':[], 'super':[],'interface':[]}

cfg = json.load(open('config/data-params.json'))
# main(**cfg)
# bg.create_edges(packageD, P_graph)
# bg.create_edges(invokeD, I_graph)
A, apis, apks = bg.buildA_matrix('./data/smali')

entertainment-146
personalization-48
game_simulation-29
game_arcade-126
lifestyle-424
travel_and_local-151
business-24
productivity-20
entertainment-47
books_and_reference-195
entertainment-70
game_board-5
productivity-19
shopping-1
game_adventure-108
travel_and_local-121
business-325
game_casual-108
business-118
maps_and_navigation-27
tools-20
education-484
entertainment-138
personalization-105
food_and_drink-89
game_puzzle-52
game_adventure-46
events-18
game_arcade-90
tools-111
sports-70
game_adventure-59
game_arcade-182
entertainment-633
game_casual-118
communication-107
tools-245
game_educational-22
game_casual-161
game_word-8
business-344
game_casual-102
events-3
libraries_and_demo-15
health_and_fitness-163
entertainment-502
photography-32
productivity-5
game_arcade-108
lifestyle-450
CPU times: user 2min 49s, sys: 14.5 s, total: 3min 4s
Wall time: 3min 37s


In [ ]:
A.shape

In [ ]:
packageD

In [ ]:
A

In [ ]:
B = nx.adjacency_matrix(B_graph, apis.values)
I = nx.adjacency_matrix(I_graph, apis.values)

In [ ]:
len(B_graph.edges)

In [ ]:
(P_graph.nodes)

In [ ]:
ABA = A*B*A.transpose()

In [ ]:
A.shape

In [ ]:
B.shape

In [ ]:
ABA.shape

In [ ]:
ABA

In [ ]:
%%time
def create_B(smali_src, **kwargs):
    smali = cdb.get_smali(smali_src)
    B_graph = nx.Graph()
    same_block = defaultdict(list)
    for f in smali:
        with open(f) as file:
            blocks = defaultdict(list)
            # build B
            bg.locate_method_blocks(file, blocks)
            for b in blocks:
                key = f + str(b)
                apis = []
                bg.find_api_calls(blocks[b], apis)
                if (len(apis) >= 2):
                    same_block[key] = apis

    bg.create_edges(same_block, B_graph)
    return B_graph

cfg = json.load(open('config/data-params.json'))
B = nx.adjacency_matrix(create_B(**cfg), apis.values)

In [ ]:
B

In [8]:
%%time
def build_P(smali_src, **kwargs):
    # initalize data
    smali = cdb.get_smali(smali_src)
    
    P_graph = nx.Graph()
    packageD = defaultdict(list)

    count = 0
    for f in smali:
        with open(f) as file:
            # build P
            for line in file:
                if('invoke-' in line and 'method' not in line): 
                    try:
                        if(count < 1000):
                            api = []
                            bg.find_api_calls([line], api)
                            package = bg.find_package(line)
                            packageD[package].append(api[0])
                            count += 1
                        else:
                            
                            bg.create_edges(packageD, P_graph)
                            packageD = defaultdict(list)
                            count = 0
                    except:
                        pass

    return  P_graph


cfg = json.load(open('config/data-params.json'))
P = build_P(**cfg)

CPU times: user 3min 21s, sys: 9.99 s, total: 3min 31s
Wall time: 3min 31s


In [9]:
nx.adjacency_matrix(P, apis.values)

<422660x422660 sparse matrix of type '<class 'numpy.int64'>'
	with 10982501 stored elements in Compressed Sparse Row format>

In [63]:
%%time
def buildI(smali_src, **kwargs):
    # initalize data
    smali = cdb.get_smali(smali_src)
    I_graph = nx.Graph()
    invokeD = {'direct': [], 'virtual':[], 'static':[], 'super':[],'interface':[]}

    for f in smali[:4000]:
        with open(f) as file:
            for line in file:
                if('invoke-' in line and 'method' not in line):
                    api = []
                    bg.find_api_calls([line], api)
                    invoke = bg.find_invoke(line)
                    invokeD[invoke].append(api[0])
    for key in invokeD:
        invokeD[key] = set(invokeD[key])
        
    bg.create_edges(invokeD, I_graph)
    return I_graph

cfg = json.load(open('config/data-params.json'))
I = buildI(**cfg)
I

CPU times: user 1min 4s, sys: 13.2 s, total: 1min 17s
Wall time: 1min 22s


422660

In [62]:
len(I.edges)

13980425

In [59]:
%%time
nx.adjacency_matrix(I, apis.values)

CPU times: user 25.9 s, sys: 1.11 s, total: 27 s
Wall time: 27.3 s


<422660x422660 sparse matrix of type '<class 'numpy.int64'>'
	with 11548092 stored elements in Compressed Sparse Row format>